## Домашнее задание «Функции и работа с данными»

In [162]:
import pandas as pd
import numpy as np

#### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

- оценка 2 и ниже — низкий рейтинг;
- оценка 4 и ниже — средний рейтинг;
- оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец **class**.

In [2]:
movies = pd.read_csv('D://ratings.csv')

In [52]:
movies.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [ ]:
# Вопрос: почему такая реализация выполняется очень долго? И как ее можно оптимизировать?

In [42]:
mean_rating = movies.groupby('movieId').mean().reset_index()[['movieId', 'rating']]

In [45]:
def classification(row):

    if float(mean_rating['rating'][mean_rating.movieId == row['movieId']]) <= 2:
        return 'низкий рейтинг'
    elif float(mean_rating['rating'][mean_rating.movieId == row['movieId']]) >= 4.5:
        return 'высокий рейтинг' 
    else: 
        return 'средний рейтинг'

In [47]:
movies['class'] = movies.apply(classification, axis = 1)

In [ ]:
# упрощенная реализация

In [50]:
def classification_simple(row):

    if float(row['rating']) <= 2:
        return 'низкий рейтинг'
    elif float(row['rating']) >= 4.5:
        return 'высокий рейтинг' 
    else: 
        return 'средний рейтинг'

In [51]:
movies['class'] = movies.apply(classification_simple, axis = 1)

#### Задание 2
Используйте файл *keywords.csv.*

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

<code>geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}</code>

Результат классификации запишите в отдельный столбец **region**.

In [58]:
keywords = pd.read_csv('D://keywords.csv')
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [103]:
def define_region(row):
    regions = []
    for region in geo_data.keys():
        if (len(set(w[:-1].lower() for w in row.keyword.split(' ')) & set(w[:-1].lower() for w in geo_data[region])) > 0) or (len(set(w[:-1].lower() for w in row.keyword.split(' ')) & set(w.lower() for w in geo_data[region])) > 0):
            regions.append(region)
    if len(regions)==0:
        return 'undefined'
    else:
        return ', '.join(regions)  
    

In [104]:
keywords['region'] = keywords.apply(define_region, axis = 1)

In [115]:
keywords[(keywords.region!= 'undefined')].head()

,keyword,shows,region
127,авито москва,979292,Центр
143,эхо москвы,889657,Центр
197,погода в москве,745745,Центр
370,авито ру санкт петербург,425134,Северо-Запад
414,погода в москве на 14 дней,400914,Центр


#### Задание 3 (бонусное)

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл *movies.csv* и *ratings.csv* из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010 года.

1. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:

    1. для каждой строки пройдите по всем годам списка years;
    1. если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
    1. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
    
1. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

1. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [257]:
rating = pd.read_csv('D://ml-25m/ml-25m/ratings.csv')
movies = pd.read_csv('D://ml-25m/ml-25m/movies.csv')

In [260]:
rating.head()

,userId,movieId,rating,timestamp,year
0,1,296,5.0,1147880044,1994
1,1,306,3.5,1147868817,1994
2,1,307,5.0,1147868828,1993
3,1,665,5.0,1147878820,1995
4,1,899,3.5,1147868510,1952


In [267]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [165]:
years = set(range(1950,2011))

In [123]:
import re

In [254]:
def production_year(Id):
    compare = list(set(np.array(re.findall(r'\((\d+)\)',str(movies.title[(int(Id) == movies.movieId)])), dtype=int)) & years)
    if len(compare) > 1:
        compare = ', '.join(compare)
    elif len(compare) == 1:
        compare = str(compare[0])
    else:
        compare = '1900'
    return compare

In [191]:
# тестируем функцию на одной строке

In [ ]:
row1 = pd.DataFrame({'userId':[1], 'movieId': [296], 'rating': [4.3], 'timestamp':[1147868510]})

In [255]:
production_year(row1.movieId)

'1994'

In [ ]:
# если не ограничить, выполняется очень долго. Подскажите, пожалуйста, как можно оптимизировать? Что в моем решении неоптимально?
rating = rating.head(500)

In [259]:
rating['year'] = rating.movieId.apply(production_year)

In [264]:
mean_rating = rating.groupby('year').mean().reset_index()[['year', 'rating']].sort_values(by='rating', ascending=False)

In [266]:
mean_rating.head(10)

,year,rating
1,1950,5.000000
7,1959,5.000000
19,1975,5.000000
3,1954,4.750000
22,1978,4.750000
18,1974,4.666667
30,1986,4.625000
27,1983,4.500000
4,1956,4.500000
6,1958,4.500000
